In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import data_utils
import download_script


In [2]:
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
download_dir = "./data"
download_script.download_file(url,download_dir)

100%|██████████| 170498071/170498071 [00:26<00:00, 6402374.32it/s] 


Download finished. Extracting files.
Extraction Done.


In [3]:
data_dir = "./data/cifar-10-batches-py"
X_train, y_train, X_test, y_test = data_utils.load_CIFAR10(data_dir)